In [4]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Assume 'tsla' is your DataFrame with a MultiIndex as shown in your output
tsla = pd.DataFrame(
    [('Close', 'TSLA'), ('High', 'TSLA'), ('Low', 'TSLA'), ('Open', 'TSLA'), ('Volume', 'TSLA')],
    columns=['Price', 'Ticker']
).set_index(['Price', 'Ticker'])

# To access 'Close' at the 'Price' level
scaler = MinMaxScaler()
tsla_scaled = scaler.fit_transform(tsla[[('Close', 'TSLA')]])

print(tsla_scaled)

KeyError: "None of [Index([('Close', 'TSLA')], dtype='object')] are in the [columns]"

In [3]:
# Assuming 'tsla' is your DataFrame
print(tsla.columns)

MultiIndex([( 'Close', 'TSLA'),
            (  'High', 'TSLA'),
            (   'Low', 'TSLA'),
            (  'Open', 'TSLA'),
            ('Volume', 'TSLA')],
           names=['Price', 'Ticker'])


In [5]:
train = tsla.loc[:'2023-12-31']
test = tsla.loc['2024-01-01':]

In [7]:
from pmdarima import auto_arima
model_arima = auto_arima(train['Adj Close'], seasonal=True, m=12)  # m=12 for monthly seasonality
print(model_arima.summary())
forecast_arima = model_arima.predict(n_periods=len(test))

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [8]:
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(train['Adj Close'], order=(p,d,q))  # Tune p,d,q via grid search
fitted = model.fit()
forecast_arima = fitted.forecast(steps=len(test))

KeyError: 'Adj Close'

In [9]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Create sequences
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        xs.append(data[i:i+seq_length])
        ys.append(data[i+seq_length])
    return np.array(xs), np.array(ys)

seq_length = 60
X_train, y_train = create_sequences(tsla_scaled[:len(train)], seq_length)
X_test, y_test = create_sequences(tsla_scaled[len(train):], seq_length)

# Convert to tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
# Similar for test

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq), 1, -1))
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

model_lstm = LSTMModel()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)

# Training loop
epochs = 50
for i in range(epochs):
    for seq, labels in zip(X_train, y_train):  # Use DataLoader for batching in practice
        optimizer.zero_grad()
        y_pred = model_lstm(seq)
        loss = loss_function(y_pred, labels)
        loss.backward()
        optimizer.step()

# Forecasting
model_lstm.eval()
forecast_lstm = []
inputs = torch.from_numpy(tsla_scaled[-seq_length:]).float()
for _ in range(len(test)):
    pred = model_lstm(inputs[-seq_length:])
    forecast_lstm.append(pred.item())
    inputs = torch.cat((inputs, pred.unsqueeze(0)))
forecast_lstm = scaler.inverse_transform(np.array(forecast_lstm).reshape(-1,1))

NameError: name 'tsla_scaled' is not defined

In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae_arima = mean_absolute_error(test['Adj Close'], forecast_arima)
rmse_arima = np.sqrt(mean_squared_error(test['Adj Close'], forecast_arima))
mape_arima = np.mean(np.abs((test['Adj Close'] - forecast_arima) / test['Adj Close'])) * 100

# Similar for LSTM
print(f'ARIMA: MAE={mae_arima}, RMSE={rmse_arima}, MAPE={mape_arima}')

KeyError: 'Adj Close'